## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: A549

In [2]:
import pandas as pd
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
cell_line='A549'

---

In [4]:
from BIOINF_tesi.data_pipe import Load_Create_Task
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline

In [5]:
from BIOINF_tesi.models import FFNN_define_model, CNN_define_model, CNN_LSTM_define_model
from BIOINF_tesi.models.utils import fit, Param_Search, Kfold_CV,get_loss_weights_from_dataloader

In [6]:
data = Load_Create_Task()
data.load(verbose=True)

Loading data:   0%|          | 0/9 [00:00<?, ?it/s]

Loading data:   0%|          | 0/10 [00:00<?, ?it/s]

enhancers files:
['A549', 'GM12878', 'H1', 'HEK293', 'HEPG2', 'K562', 'MCF7', 'bed', 'fa']

bed has shape: (63285, 11)
MCF7 has shape: (63285, 121)
GM12878 has shape: (63285, 156)
A549 has shape: (63285, 52)
H1 has shape: (63285, 62)
K562 has shape: (63285, 433)
fa has shape: (63285, 4)
HEPG2 has shape: (63285, 566)
HEK293 has shape: (63285, 200)

promoters files:
['A549', 'GM12878', 'H1', 'HEK293', 'HEPG2', 'K562', 'MCF7', 'bed', 'fa']

MCF7 has shape: (99881, 121)
fa has shape: (99881, 4)
GM12878 has shape: (99881, 156)
A549 has shape: (99881, 52)
H1 has shape: (99881, 62)
K562 has shape: (99881, 433)
HEPG2 has shape: (99881, 566)
bed has shape: (99881, 11)
HEK293 has shape: (99881, 200)


In [6]:
%store data

Stored 'data' (Load_Create_Task)


In [5]:
%store -r data

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [7]:
data_dict, labels_dict = data.get_task('active_E_vs_inactive_E')
pipe_data_load = Build_DataLoader_Pipeline(data_dict, labels_dict, path_name='active_E_vs_inactive_E.pickle', verbose=False)

Data Preprocessing Done!


In [9]:
import pickle
with open('pipe_data_load.pickle', 'wb') as handle:
    pickle.dump(pipe_data_load, handle)

### 1. EPIGENOMIC FEATURES

In [7]:
model = FFNN_define_model

In [8]:
kf_CV = Kfold_CV()

kf_CV.run(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                augmentation=False,
                model = model,
                study_name = 'prova_lstm',
                hp_model_path = 'provaa',
                test_model_path = 'provaaw')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:295: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler=BoTorchSampler())
[I 2021-07-22 09:52:42,365] Using an existing study with name 'prova_lstm' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
g_gpu = torch.Generator(device)

In [13]:
random_state=123

In [14]:
g_gpu.manual_seed(random_state+30)

### 2. GENOME SEQUENCE

In [ ]:
model = CNN_define_model

kf_CV = Kfold_CV()

kf_CV.run(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 1,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                augmentation=False,
                model = model,
                study_name = 'prova_lstm',
                hp_model_path = 'provaa',
                test_model_path = 'provaaw')

---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [ ]:
data_dict, labels_dict = data.get_task('active_P_vs_inactive_P')
pipe_data_load = Build_DataLoader_Pipeline(data_dict, labels_dict, path_name='active_P_vs_inactive_P.pickle', verbose=False)

---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [ ]:
data_dict, labels_dict = data.get_task('active_E_vs_active_P')
pipe_data_load = Build_DataLoader_Pipeline(data_dict, labels_dict, path_name='active_E_vs_active_P.pickle', verbose=False)

---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [ ]:
data_dict, labels_dict = data.get_task('inactive_E_vs_inactive_P')
pipe_data_load = Build_DataLoader_Pipeline(data_dict, labels_dict, path_name='inactive_E_vs_inactive_P.pickle', verbose=False)

---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [ ]:
data_dict, labels_dict = data.get_task('active_EP_vs_inactive_rest')
pipe_data_load = Build_DataLoader_Pipeline(data_dict, labels_dict, path_name='active_EP_vs_inactive_rest.pickle', verbose=False)